In [1]:
from gurobipy import Model, quicksum, GRB

In [2]:
Plnt = {'plnt'}
DC = set(range(1,3))
Cust = DC.copy()
TRoute = {(i,j) for i in Plnt for j in DC}
DRoute = {(j,k) for j in DC for k in Cust}
Route = TRoute | DRoute
CT = {1,2}

T = 3
Period = list(range(1, T+1))

In [3]:
tp_cost = {(i,j):5 for i,j in TRoute}
del_cost = {(j, k):10 for j,k in DRoute}
demand = {(k,t):10 for k in Cust for t in Period}
dc_run_cost = {(j,ct):1000 for j in DC for ct in CT}
dc_inv_cost = {j:5 for j in DC}
cust_inv_cost = {k:30 for k in Cust}

In [4]:
model = Model()

Academic license - for non-commercial use only


In [5]:
x, y, dc_inv, dc_plus_inv, cust_inv, cust_plus_inv = {}, {}, {}, {}, {}, {}

for i,j in Route:
    for t in Period:
        x[i,j,t] = model.addVar(vtype="C", name=f'x[{i},{j},{t}]')

for j in DC:
    for ct in CT:
        for t in Period:
            y[j,ct,t] = model.addVar(vtype="B", name=f'y[{j},{ct},{t}]')
    
for j in DC:
    for t in Period:
        dc_inv[j,t] = model.addVar(vtype="C", name=f'dc_inv[{j},{t}]')
    
for j in DC:
    dc_plus_inv[j] = model.addVar(vtype="C", name=f'dc_plus_inv[{j}]')

for k in Cust:
    for t in Period:
        cust_inv[k,t] = model.addVar(vtype="C", name=f'cust_inv[{k},{t}]')

for k in Cust:
    cust_plus_inv[k] = model.addVar(vtype="C", name=f'cust_plus_inv[{k}]')

model.update()

In [6]:
Cust_Demand_Cons, DC_Running_Cons, DC_Flow_Cons, Center_Type_Cons = {}, {}, {}, {}

for k in Cust:
    for t in Period:
        Cust_Demand_Cons[k,t] = model.addConstr(
            quicksum(x[j,k,t] for j in DC)
            + (0 if t==1 else cust_inv[k,t-1] )
            ==
            demand[k,t]
            + (cust_plus_inv[k] if t==T else cust_inv[k,t])
        )

for j,k in DRoute:
    for t in Period:
        DC_Running_Cons[j,k,t] = model.addConstr(
            x[j,k,t]
            <=
            quicksum(demand[k,t_] for t_ in Period) * quicksum(y[j,ct,t] for ct in CT) 
        )

for j in DC:
    for t in Period:
        DC_Flow_Cons[j,t] = model.addConstr(
            quicksum(x[i,j,t] for i in Plnt)
            + (0 if t==1 else dc_inv[j,t-1] )
            ==
            quicksum(x[j,k,t] for k in Cust)
            + (dc_plus_inv[j] if t==T else dc_inv[j,t])
        )

for j in DC:
    for t in Period:
        Center_Type_Cons[j] = model.addConstr(
            quicksum(y[j,ct,t] for ct in CT)
            <=
            1
        )
    
    
model.update()

In [7]:
model.setObjective(
    quicksum(tp_cost[i,j] * x[i,j,t] for i,j in TRoute for t in Period) +
    quicksum(del_cost[j,k] * x[j,k,t] for j,k in DRoute for t in Period) +
    quicksum(dc_run_cost[j,ct] * y[j,ct,t] for j in DC for ct in CT for t in Period) +
    quicksum(dc_inv_cost[j] * dc_inv[j,t] for j in DC for t in Period) +
    quicksum(dc_inv_cost[j] * dc_plus_inv[j] for j in DC) + 
    quicksum(cust_inv_cost[k] * cust_inv[k,t] for k in Cust for t in Period) +
    quicksum(cust_inv_cost[k] * cust_plus_inv[k] for k in Cust)
    ,GRB.MINIMIZE
)
model.update()

In [8]:
model.optimize()

Optimize a model with 30 rows, 46 columns and 98 nonzeros
Variable types: 34 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 4100.0000000
Presolve removed 21 rows and 32 columns
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 22 nonzeros
Variable types: 8 continuous, 6 integer (6 binary)

Root relaxation: objective 2.500000e+03, 9 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 2500.00000    0    2 4100.00000 2500.00000  39.0%     -    0s
H    0     0                    3500.0000000 2500.00000  28.6%     -    0s
     0     0 3400.00000    0    1 3500.00000 3400.00000  2.86%     -    0s
     0     0 3400.00000    0    3 3500.00000 3400.00000  2.86%     -    0s
   

In [9]:
for v in model.getVars():
    if v.X > 0:
        print(v.VarName, v.X)

x[1,2,1] 10.0
x[plnt,2,2] 40.0
x[2,2,2] 20.0
x[plnt,1,1] 20.0
x[1,1,1] 10.0
x[2,1,2] 20.0
y[1,1,1] 1.0
y[2,2,2] 1.0
cust_inv[1,2] 10.0
cust_inv[2,2] 10.0
